### Задача №1

Камера повернута относительно мировой системы координат на 45
градусов вокруг оси z и сдвинута на 10 вдоль оси z мировой системы координат.
Внутренние параметры камеры: fx=fy=400, cx=960, cy=540. Написать программу,
которая выводит на экран матрицу проекции камеры и целочисленные
координаты пикселя на изображении, соответствующего трехмерной точке с
координатами (10, -10, 100) в мировой системе координат.


$$ w\begin{pmatrix} u\\\ v\\\ 1 \end{pmatrix} = P\begin{pmatrix} X_0 \\\
Y_0 \\\ Z_0 \\\ 1 \end{pmatrix} $$

$$ P = K[R|T] $$

$$ K = \begin{pmatrix} f_x & 0 & c_x \\\
 0 & f_y & c_x\\\ 0 & 0 & 1 \end{pmatrix}

In [15]:
import numpy as np
import math

K = np.matrix([[400, 0, 960],
                [0, 400, 540],
                [0, 0, 1]])

RT = np.matrix([[math.cos(math.pi/4), -math.sin(math.pi/4), 0, 0],
                [math.sin(math.pi/4), math.cos(math.pi/4), 0, 0],
                [0, 0, 1, 10]])

P = K.dot(RT)

print(f"Матрица проекции камеры:\n{P}")

Матрица проекции камеры:
[[ 2.82842712e+02 -2.82842712e+02  9.60000000e+02  9.60000000e+03]
 [ 2.82842712e+02  2.82842712e+02  5.40000000e+02  5.40000000e+03]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  1.00000000e+01]]


In [16]:
XYZ1 = np.matrix([[10],
                [-10],
                [100],
                [1]])

uvw = P.dot(XYZ1)

print(f"Новые координаты пикселя:\nu={int(uvw[0]/uvw[2])}\nv={int(uvw[1]/uvw[2])}")

Новые координаты пикселя:
u=1011
v=540


### Задача №2

Написать генератор пар двумерных точек для теста алгоритма,
удовлетворяющих заданному заранее преобразованию гомографии H0. Используя
прямое линейное преобразование (DLT) и SVD, написать программу, которая
принимает на вход множество пар двумерных точек, и находит соответствующую
этим парам матрицу гомографии H. Распечатать матрицы H0 и H.

In [17]:
import random
import cv2
random.seed(10)

def generate_homogramm_pair(h0: np.matrix)-> tuple:
    u0, v0 = random.randint(0, 100), random.randint(0, 100)
    u0v0 = np.matrix([[u0],
                    [v0],
                    [1]])
    uvw = h0.dot(u0v0)
    return ((u0, v0), (float(uvw[0]/uvw[2]), float(uvw[1]/uvw[2])))


h0 = np.matrix([[1, 7, 3],
                [4, 5, 4],
                [2, 1, 9]])

generate_homogramm_pair(h0)

((73, 4), (0.6540880503144654, 1.9874213836477987))

In [18]:
l1 = []
l2 = []
for i in range(100):
    points = generate_homogramm_pair(h0)
    l1.append(points[0])
    l2.append(points[1])

In [19]:
def find_homography(points_source, points_target):
    A  = construct_A(points_source, points_target)
    u, s, vh = np.linalg.svd(A, full_matrices=True)
    homography = vh[-1].reshape((3,3))
    return homography/homography[2,2]

def construct_A(points_source, points_target):
    assert points_source.shape == points_target.shape, "Shape does not match"
    num_points = points_source.shape[0]

    matrices = []
    for i in range(num_points):
        partial_A = construct_A_partial(points_source[i], points_target[i])
        matrices.append(partial_A)
    return np.concatenate(matrices, axis=0)

def construct_A_partial(point_source, point_target):
    x, y, z = point_source[0], point_source[1], 1
    x_t, y_t, z_t = point_target[0], point_target[1], 1

    A_partial = np.array([
        [0, 0, 0, -z_t*x, -z_t*y, -z_t*z, y_t*x, y_t*y, y_t*z],
        [z_t*x, z_t*y, z_t*z, 0, 0, 0, -x_t*x, -x_t*y, -x_t*z]
    ])
    return A_partial



In [20]:
h = find_homography(np.array(l1), np.array(l2))
h

array([[0.11111111, 0.77777778, 0.33333333],
       [0.44444444, 0.55555556, 0.44444444],
       [0.22222222, 0.11111111, 1.        ]])

In [21]:
h, status = cv2.findHomography(np.array(l1), np.array(l2))
h

array([[0.11111109, 0.77777763, 0.33333323],
       [0.44444436, 0.55555544, 0.44444494],
       [0.22222218, 0.11111109, 1.        ]])

In [22]:
h0

matrix([[1, 7, 3],
        [4, 5, 4],
        [2, 1, 9]])

Как видно, матрицы отличаются с точностью до константы (ну почти)

### Задача №3

Модифицировать генератор из Задачи 7 так, чтобы он выдавал n=1000
соответствий, из которых 30% соответствуют заданному заранее преобразованию
гомографии, а остальные взяты случайно (преобразованию гомографии не
удовлетворяют). Реализовать нахождение преобразования гомографии H1,
используя DLT и RANSAC. Распечатать H0 и H1.


In [23]:
def generate_homogramm_pairs(h0: np.matrix, n=1000)-> tuple:
    true_part = int(n*0.3)
    false_part = n - true_part
    l1 = []
    l2 = []
    for i in range(true_part):
        u0, v0 = random.randint(0, 100), random.randint(0, 100)
        u0v0 = np.matrix([[u0],
                        [v0],
                        [1]])
        uvw = h0.dot(u0v0)
        l1.append((u0, v0))
        l2.append((float(uvw[0]/uvw[2]), float(uvw[1]/uvw[2])))
    for i in range(false_part):
        l1.append((random.randint(0, 100), random.randint(0, 100)))
        l2.append((random.randint(0, 100), random.randint(0, 100)))
    return np.array(l1), np.array(l2)

In [24]:
h0 = np.matrix([[1, 7, 3],
                [4, 5, 4],
                [2, 1, 9]])

generate_homogramm_pairs(h0)

(array([[73, 80],
        [61,  3],
        [14, 37],
        ...,
        [14, 96],
        [80, 55],
        [33, 99]]),
 array([[ 2.70638298,  2.96170213],
        [ 0.63432836,  1.96268657],
        [ 3.72972973,  3.31081081],
        ...,
        [86.        , 84.        ],
        [80.        , 37.        ],
        [46.        , 99.        ]]))

In [25]:
pairs = generate_homogramm_pairs(h0)
h, status = cv2.findHomography(pairs[0], pairs[1], method=cv2.RANSAC)
h

array([[ 0.395091  ,  2.50647403, -3.19198375],
       [ 1.43239166,  1.82230682, -2.72099647],
       [ 0.71073998,  0.38625573,  1.        ]])

In [26]:
h0

matrix([[1, 7, 3],
        [4, 5, 4],
        [2, 1, 9]])

### Задача №4

Два изображения одной и той же сцены сделаны одной и той же
камерой с такой же матрицей внутренних параметров, как в Задаче 6. Второй
снимок сделан после поворота камеры на 30 градусов вокруг оси x относительно
начала координат системы отсчета, связанной с камерой (без сдвига). Найти
матрицу гомографии между двумя изображениями.

In [27]:
K = np.matrix([[400, 0, 960],
                [0, 400, 540],
                [0, 0, 1]])

RT1 = np.matrix([[1, 0, 0, 0],
                [0, math.cos(0), -math.sin(0), 0],
                [0, math.sin(0), math.cos(0), 0]])

RT2 = np.matrix([[1, 0, 0, 0],
                [0, math.cos(math.pi/6), -math.sin(math.pi/6), 0],
                [0, math.sin(math.pi/6), math.cos(math.pi/6), 0]])

P1 = K.dot(RT1)
P2 = K.dot(RT2)

Для нахождения матрицы гомографии нужно по крайней мере 4 точки:

In [28]:
XYZ1 = np.matrix([[10],
                [10],
                [100],
                [1]])
XYZ2 = np.matrix([[10],
                [-10],
                [100],
                [1]])
XYZ3 = np.matrix([[-10],
                [10],
                [100],
                [1]])
XYZ4 = np.matrix([[-10],
                [-10],
                [100],
                [1]])

points_3d = [XYZ1, XYZ2, XYZ3, XYZ4]
l1 = np.array([P1.dot(i) for i in points_3d])
l2 = np.array([P1.dot(i) for i in points_3d])

In [29]:
h, status = cv2.findHomography(l1, l2)
print(f"Матрица гомографии между двумя изображениями:\n{h}")

Матрица гомографии между двумя изображениями:
[[ 1.00000000e+00  0.00000000e+00 -1.96911379e-13]
 [ 0.00000000e+00  1.00000000e+00 -2.95367069e-13]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
